In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%autosave 10

gpu_id='gpu2'
import os
os.environ["THEANO_FLAGS"] = "device="+gpu_id +",lib.cnmem=1"

import numpy as np
from keras.layers.advanced_activations import SReLU
from keras import callbacks as ckbs
import random
import time
import gzip
from keras.models import model_from_json
import pickle as pkl
import keras
from keras.models import Sequential
from keras.layers.core import *
from keras.layers.wrappers import *
from keras.optimizers import *
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.layers import LSTM
from docutils.languages.af import labels
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import gensim

Autosaving every 10 seconds


Using Theano backend.
Using gpu device 2: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 4007)
/home/sag/src_George/workspace/virtualenv/lib/python3.4/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
f = gzip.open('../data/processed/pkl_dialog_select/embeddings.pkl.gz', 'rb')
embeddings = pkl.load(f)
f.close()

label2Idx = embeddings['label2Idx']

#Inverse label mapping
idx2Label = {v: k for k, v in label2Idx.items()}

f = gzip.open('../data/processed/pkl_dialog_select/data.pkl.gz', 'rb')
test_data = pkl.load(f)
dev_data = pkl.load(f)
#wiki_data = pkl.load(f)
f.close()

n_out = len(label2Idx)

In [3]:
def get_data_for_testing(matrix, target):
   
    train_samples = int(matrix.shape[0]*1)
    X_train = matrix[:train_samples]
    X_test = matrix[train_samples:]
        
    y_train = target[:train_samples]
    y_test = target[train_samples:]
        
    
    
    return  [X_train[:,:,i] for i in range(X_train.shape[2])], \
            y_train

In [4]:
def load_neural_network(file_from):
    (nn_arch, nn_weights_path) = pkl.load(open(file_from, 'rb'))
    nn = model_from_json(nn_arch)
    nn.set_weights(nn_weights_path)
    return nn

In [5]:
def get_data(data):
    print( "%d test size" % len(data))
    data = np.array(data)
    print(data.shape)
    X = [x[0] for x in data]
    y = [x[1][0] for x in data]
    return np.array(X), np.array(y)

In [6]:
def divide_target(y): 
    y_t = [] 
    for x in y: 
        y_t.append(x[0]) 
    return np.array(y_t)

In [7]:
from sklearn.metrics import classification_report
X_test,y_t = get_data(test_data)

59381 test size
(59381, 2)


In [8]:
y_test = divide_target(y_t)

In [9]:
X_test,y_test = get_data_for_testing(X_test,y_test)

In [ ]:
y_test = y_test.astype(np.int64)

This function load model and make test (classification report)

In [ ]:
def process(model_name):
    model = load_neural_network('../models/' + model_name)
    pred = model.predict_classes(X_test, batch_size=128, verbose=1)
    target_names = [idx2Label[0], idx2Label[1], idx2Label[2], idx2Label[3]]
    print(classification_report(y_test, pred, target_names=target_names))
    return model

In [ ]:
idx2Label[3]

In [ ]:
process('new1')

In [ ]:
process('new4')

In [ ]:
process('new2')

In [ ]:
m = process('new5')

In [ ]:
m.summary()

In [ ]:
m = process('new7')

In [ ]:
m = process('new8')

In [ ]:
m = process('new12')

In [ ]:
m = process('new13')

In [ ]:
m = process('new14')

In [ ]:
m = process('new16')

In [ ]:
m = process('new17')

In [10]:
y_t[0]

array(['1', 'book_3539.task1', '0', '9'], 
      dtype='<U21')

In [11]:
model = load_neural_network('../models/' + 'new17')
pred = model.predict_classes(X_test, batch_size=128, verbose=1)


59381/59381 [==============================] - 9s     


In [ ]:
save_t = []
for x in y_t:
    save_t.append(x)

In [17]:
X_test,y_t = get_data(test_data)
for x in range(0, len(y_t)):
    y_t[x][0] = idx2Label[int(pred[x])].lower()

59381 test size
(59381, 2)


In [ ]:
file_name = y_t[0][1]
f = open("./pred/" + file_name, "w+")
pr = y_t[0]
kol = -1
for y in y_t[1:]:
    kol = kol + 1
    
    if (pr[0] == y[0]) and (int(pr[2]) + int(pr[3]) + 1 == int(y[2])):
        
        y[3] = str(int(pr[3]) + 1 + int(y[3]))
        y[2] = pr[2]
        pr = y
    elif (pr[0] == y[0]) and (int(pr[2]) + int(pr[3]) + 2 == int(y[2])):
        
        y[3] = str(int(pr[3]) + 2 + int(y[3]))
        y[2] = pr[2]
        pr = y
        print(pr)
    else:
        if pr[0] != '3':
            s = idx2Label[int(pr[0])].lower() + " " + pr[2] + " " + pr[3] + "\n"
            f.write(s)
        pr = y
    if file_name != y[1]:
        f.close()
        file_name = y[1]
        f = open("./pred/" + file_name, "w+")
        #break
s = idx2Label[int(pr[0])].lower() + " " + pr[2] + " " + pr[3] + "\n"
if pr[0] != '1': 
    f.write(s)
f.close()

In [26]:
import pandas as pd
allINeed = pd.DataFrame(y_t, columns=["mark", "FileName", "posStartInText", "len"])
feats = pd.read_csv('../data/processed/new/Dialog_test/NER_testset.features.csv',sep=';',encoding='utf8')
targets = pd.read_csv('../data/processed/new/Dialog_test/NER_testset.targets.csv',sep=';',encoding='utf8')

In [29]:
 def WriteLinesTo(allObjects, fileName):
    f = open(fileName, "w+")
    #print(len(allObjects))
    f.writelines(allObjects)
    f.close()
    

In [33]:
#allINeed = pd.concat([feats.loc[:,["posStartInText","len"]],targets.loc[:,["mark","FileName"]]],axis=1)

tags = {"per":"per","loc":"loc","org":"org"}
#tags = ["per", "loc","org"]
for fileName in list(set(allINeed["FileName"])):
    allObjects = []
    currentObject = []
    for cl in tags.keys():
        for i,row in allINeed.loc[targets["FileName"]==fileName].iterrows():
            if row["mark"]==cl:
                if len(currentObject)>0:
                    lengthOfObj = int(row['posStartInText']) - int(currentObject[1]) + int(row['len'])
                    currentObject[2] = lengthOfObj
                else:
                    currentObject.append(tags[cl])
                    currentObject.append(row['posStartInText'])
                    currentObject.append(row['len'])
            else:
                if len(currentObject)>0:
                    currentObject[1]=str(int(currentObject[1]))
                    currentObject[2]=str(int(currentObject[2]))
                    allObjects.append(currentObject)
                    currentObject = []
        if len(currentObject)>0:
            currentObject[1]=str(int(currentObject[1]))
            currentObject[2]=str(int(currentObject[2]))
            allObjects.append(currentObject)
            currentObject = []
    allObjects = [" ".join(obj)+"\n" for obj in allObjects]
    WriteLinesTo(allObjects,"./forTests/"+fileName)

24
71
22
58
29
53
14
16
68
18
8
37
31
29
14
39
76
30
46
48
32
10
52
25
35
18
21
19
21
21
48
16
24
62
290
28
21
34
15
127
26
40
226
38
64
40
31
22
43
17
70
24
8
12
26
30
22
25
22
57
62
25
27
117
44
36
17
13
32
66
37
19
53
38
29
51
29
27
19
23
62
22
30
49
79
14
12
34
9
26
21
35
30
38
18
40
55
42
33
53
28
32
35
27
94
37
96
26
54
22
29
144
56
33
18
14
12
28
20
59
69
12
35
33
51
41
61
33
36
19
29
33
